# KG3D TorchicTab Demo 

This demo...

### Visualize Dataset

In [1]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [13]:
# Open aircraft csv 
aircraft_file_path = 'sfo-aircraft-tail-numbers-and-models.csv'
lines_to_keep = 3
df = pd.read_csv(aircraft_file_path)
df = df.drop(df.columns[0], axis=1).head(lines_to_keep)[["aircraft_model", "airline", "creation_date", "tail_number"]]
aircraft_file_path_simple = 'aircraft_models.csv'
df.to_csv(aircraft_file_path_simple, index=False)
df


,aircraft_model,airline,creation_date,tail_number
0,CRJ-700,Skywest Airlines,2009-09-10T16:33:00.000,N637CA
1,E175-,Skywest Airlines,2018-04-23T08:34:00.000,N205SY
2,B747-400,KLM Royal Dutch Airlines,2018-04-14T16:49:00.000,PHBFB


### Produce Semantic Annotations with TorchicTab

In [14]:
from torchic_tab_heuristic.task_calculation_wd import generate_annotations

(subject_column, 
 primary_annotations, 
 secondary_annotations, 
 cea, 
 cpa, 
 cta) = generate_annotations(aircraft_file_path_simple)

TorchicTab Initiated!
Connecting with Elasticsearch...
Elasticsearch connection succesful!
Loading table...
Table: aircraft_models.csv 
   aircraft_model                   airline creation_date tail_number
0        CRJ-700          Skywest Airlines    2009-09-10      N637CA
1          E175-          Skywest Airlines    2018-04-23      N205SY
2       B747-400  KLM Royal Dutch Airlines    2018-04-14       PHBFB 

Calculating structural annotations...
Calculating semantic annotations...
{0: [<torchic_tab_heuristic.entity_classes.CandidateEntity object at 0x7fcce4294e20>, <torchic_tab_heuristic.entity_classes.CandidateEntity object at 0x7fcce429a550>, <torchic_tab_heuristic.entity_classes.CandidateEntity object at 0x7fcce4291910>], 1: [<torchic_tab_heuristic.entity_classes.CandidateEntity object at 0x7fcc909a1250>, <torchic_tab_heuristic.entity_classes.CandidateEntity object at 0x7fcce2e61820>, <torchic_tab_heuristic.entity_classes.CandidateEntity object at 0x7fcce2d43550>], 2: [<torchic_t

In [16]:
# Get subject column of table
print(f"The subject column of the table is column: \n {df.iloc[:, subject_column]}")

The subject column of the table is column: 
 0     CRJ-700
1       E175-
2    B747-400
Name: aircraft_model, dtype: object


In [17]:
# Get generic semantic annotations of table
print("Generic column annotations:")
for i in range(len(df.columns)):
    print("\n",df.columns[i])
    print(df.iloc[:, i])
    print(f"Generic semantic type: {secondary_annotations[i]}")


 aircraft_model
0     CRJ-700
1       E175-
2    B747-400
Name: aircraft_model, dtype: object
Generic semantic type: NE

 airline
0            Skywest Airlines
1            Skywest Airlines
2    KLM Royal Dutch Airlines
Name: airline, dtype: object
Generic semantic type: ORG

 creation_date
0    2009-09-10T16:33:00.000
1    2018-04-23T08:34:00.000
2    2018-04-14T16:49:00.000
Name: creation_date, dtype: object
Generic semantic type: DATE

 tail_number
0    N637CA
1    N205SY
2     PHBFB
Name: tail_number, dtype: object
Generic semantic type: NE


In [20]:
# Get column type annotations
print("Column type annotations:")
for type_annotation in cta:
    column = int(type_annotation[0])
    annotation = type_annotation[1]
    print("\n",df.columns[column])
    print(df.iloc[:, column])
    print(f"Generic semantic type: {secondary_annotations[column]}")
    print(f"Wikidata retrieved annotation: {annotation}")


 aircraft_model
0     CRJ-700
1       E175-
2    B747-400
Name: aircraft_model, dtype: object
Generic semantic type: NE
Wikidata retrieved annotation: http://www.wikidata.org/entity/Q15056993

 airline
0            Skywest Airlines
1            Skywest Airlines
2    KLM Royal Dutch Airlines
Name: airline, dtype: object
Generic semantic type: ORG
Wikidata retrieved annotation: http://www.wikidata.org/entity/Q46970

 tail_number
0    N637CA
1    N205SY
2     PHBFB
Name: tail_number, dtype: object
Generic semantic type: NE
Wikidata retrieved annotation: http://www.wikidata.org/entity/Q1821082


In [32]:
# Get column relationship annotations
print("Column relationships:")
for rel_annotation in cpa:
    if rel_annotation[2][-3:] == "/P1": continue #unkown relationship
    col1 = int(rel_annotation[0])
    col2 = int(rel_annotation[1])
    relationship = rel_annotation[2]
    print(f"\nColumn: {df.columns[col1]} --> {relationship} --> Column: {df.columns[col2]}")
    

Column relationships:

Column: aircraft_model --> http://www.wikidata.org/prop/direct/P137 --> Column: airline


In [33]:
# Get cell annotations
print("Cell entity annotations:")
for cell_annotation in cea:
    col = int(cell_annotation[0])
    row = int(cell_annotation[1])
    annotation = cell_annotation[2]
    print(f"\nColumn: {df.columns[col]}")
    print(f"Cell name: '{df.iloc[row, col]}' --> Annotation: {annotation}")


Cell entity annotations:

Column: aircraft_model
Cell name: 'CRJ-700' --> Annotation: http://www.wikidata.org/entity/Q962677

Column: aircraft_model
Cell name: 'E175-' --> Annotation: http://www.wikidata.org/entity/Q5322348

Column: aircraft_model
Cell name: 'B747-400' --> Annotation: http://www.wikidata.org/entity/Q906937

Column: airline
Cell name: 'Skywest Airlines' --> Annotation: http://www.wikidata.org/entity/Q773662

Column: airline
Cell name: 'Skywest Airlines' --> Annotation: http://www.wikidata.org/entity/Q773662

Column: airline
Cell name: 'KLM Royal Dutch Airlines' --> Annotation: http://www.wikidata.org/entity/Q181912

Column: tail_number
Cell name: 'N637CA' --> Annotation: http://www.wikidata.org/entity/Q4356221

Column: tail_number
Cell name: 'N205SY' --> Annotation: http://www.wikidata.org/entity/Q2443635

Column: tail_number
Cell name: 'PHBFB' --> Annotation: http://www.wikidata.org/entity/Q21044913
